In [12]:
import numpy as np
from collections import defaultdict


class MatrixMultiplicationMR:
    def __init__(self):
        self.intermediate = defaultdict(list)
    

    def mapper_A(self, matrix_A, p):
        """Mapper для матрицы A"""
        m, n = matrix_A.shape
        for i in range(m):
            for j in range(n):
                for k in range(p):
                    yield (i, k), ('A', j, matrix_A[i, j])
    

    def mapper_B(self, matrix_B, m):
        """Mapper для матрицы B"""
        n, p = matrix_B.shape
        for j in range(n):
            for k in range(p):
                for i in range(m):
                    yield (i, k), ('B', j, matrix_B[j, k])
    

    def shuffler(self, mapper_output):
        """Shuffler - группирует значения по ключам"""
        for key, value in mapper_output:
            self.intermediate[key].append(value)
    

    def reducer(self, key):
        """Reducer для вычисления одного элемента результирующей матрицы"""
        values = self.intermediate[key]
        
        # Создаем словари для значений A и B
        A_vals = {}
        B_vals = {}
        
        for value in values:
            matrix_type, j, val = value
            if matrix_type == 'A':
                A_vals[j] = val
            else:
                B_vals[j] = val
        
        # Вычисляем сумму произведений
        result = 0
        for j in A_vals:
            if j in B_vals:
                result += A_vals[j] * B_vals[j]
        
        return key, result
    

    def multiply(self, matrix_A, matrix_B):
        """Основной метод умножения матриц"""
        m, n1 = matrix_A.shape
        n2, p = matrix_B.shape
        
        if n1 != n2:
            raise ValueError("Несовместимые размеры матриц")
        
        # Шаг 1: Mapper для A
        mapper_a_output = self.mapper_A(matrix_A, p)
        
        # Шаг 2: Mapper для B
        mapper_b_output = self.mapper_B(matrix_B, m)
        
        # Шаг 3: Shuffler объединяет вывод обоих mapper'ов
        for item in mapper_a_output:
            self.shuffler([item])
        for item in mapper_b_output:
            self.shuffler([item])
        
        # Шаг 4: Reducer для каждого ключа
        result_matrix = np.zeros((m, p))
        for key in self.intermediate:
            i, k = key
            _, value = self.reducer(key)
            result_matrix[i, k] = value
        
        return result_matrix


# Тестирование произведения матриц
def test_matrix_multiplication():
    print("=== Тестирование произведения матриц ===")
    
    # Создаем тестовые матрицы
    A = np.array([[1, 2], [3, 4], [5, 6]])
    B = np.array([[7, 8], [9, 10]])
    
    print("Матрица A:")
    print(A)
    print("Матрица B:")
    print(B)
    
    # Умножение через MapReduce
    mr = MatrixMultiplicationMR()
    result_mr = mr.multiply(A, B)
    print("Результат MapReduce:")
    print(result_mr)
    
    # Проверка через numpy
    result_np = A @ B
    print("Результат numpy:")
    print(result_np)
    print("Результаты совпадают:", np.allclose(result_mr, result_np))


In [13]:
class LinearRegressionMR:
    def __init__(self):
        self.intermediate = defaultdict(list)
    

    def mapper_XTX_XTY(self, X, y):
        """Mapper для вычисления XᵀX и Xᵀy"""
        # Добавляем столбец единиц для intercept
        X_with_intercept = np.column_stack([np.ones(X.shape[0]), X])
        
        # Вычисляем XᵀX и Xᵀy для этой части данных
        XTX_local = X_with_intercept.T @ X_with_intercept
        XTy_local = X_with_intercept.T @ y
        
        yield 'XTX_XTY', (XTX_local, XTy_local)
    

    def reducer_XTX_XTY(self, key, values):
        """Reducer для агрегации XᵀX и Xᵀy"""
        XTX_total = None
        XTy_total = None
        
        for XTX, XTy in values:
            if XTX_total is None:
                XTX_total = XTX
                XTy_total = XTy
            else:
                XTX_total += XTX
                XTy_total += XTy
        
        return XTX_total, XTy_total
    

    def fit_normal_equation(self, X, y):
        """Обучение через нормальное уравнение"""
        # Собираем данные от всех mapper'ов
        mapper_output = list(self.mapper_XTX_XTY(X, y))
        
        # Shuffler
        for key, value in mapper_output:
            self.intermediate[key].append(value)
        
        # Reducer
        XTX_total, XTy_total = self.reducer_XTX_XTY('XTX_XTY', self.intermediate['XTX_XTY'])
        
        # Вычисляем коэффициенты
        try:
            theta = np.linalg.inv(XTX_total) @ XTy_total
        except np.linalg.LinAlgError:
            # Если матрица вырожденная, используем псевдообратную
            theta = np.linalg.pinv(XTX_total) @ XTy_total
        
        return theta


    def compute_cost(self, X, y, theta):
        """Вычисление функции стоимости"""
        X_with_intercept = np.column_stack([np.ones(X.shape[0]), X])
        predictions = X_with_intercept @ theta
        errors = predictions - y
        cost = np.sum(errors ** 2) / (2 * len(y))
        return cost
    

    def predict(self, X, theta):
        """Предсказание"""
        X_with_intercept = np.column_stack([np.ones(X.shape[0]), X])
        return X_with_intercept @ theta


# Тестирование линейной регрессии
def test_linear_regression():
    print("\n=== Тестирование линейной регрессии ===")
    
    # Генерируем тестовые данные
    np.random.seed(42)
    n_samples = 1000
    X = np.random.randn(n_samples, 2)
    true_theta = np.array([2, 3, -1])  # [intercept, coef1, coef2]
    y = true_theta[0] + X @ true_theta[1:] + 0.1 * np.random.randn(n_samples)
    
    print(f"Истинные коэффициенты: {true_theta}")
    
    # Тестируем нормальное уравнение
    print("\n1. Нормальное уравнение:")
    lr_mr = LinearRegressionMR()
    theta_normal = lr_mr.fit_normal_equation(X, y)
    print(f"Найденные коэффициенты: {theta_normal}")
    print(f"Средняя ошибка: {np.mean(np.abs(theta_normal - true_theta)):.6f}")
    
    # Сравнение с sklearn
    from sklearn.linear_model import LinearRegression
    print("\n3. Сравнение с sklearn:")
    lr_sklearn = LinearRegression()
    lr_sklearn.fit(X, y)
    theta_sklearn = np.concatenate([[lr_sklearn.intercept_], lr_sklearn.coef_])
    print(f"Sklearn коэффициенты: {theta_sklearn}")
    print(f"Средняя ошибка: {np.mean(np.abs(theta_sklearn - true_theta)):.6f}")


# Демонстрация работы
if __name__ == "__main__":
    test_matrix_multiplication()
    test_linear_regression()


=== Тестирование произведения матриц ===
Матрица A:
[[1 2]
 [3 4]
 [5 6]]
Матрица B:
[[ 7  8]
 [ 9 10]]
Результат MapReduce:
[[ 25.  28.]
 [ 57.  64.]
 [ 89. 100.]]
Результат numpy:
[[ 25  28]
 [ 57  64]
 [ 89 100]]
Результаты совпадают: True

=== Тестирование линейной регрессии ===
Истинные коэффициенты: [ 2  3 -1]

1. Нормальное уравнение:
Найденные коэффициенты: [ 2.00039354  3.0027767  -0.99828491]
Средняя ошибка: 0.001628

3. Сравнение с sklearn:
Sklearn коэффициенты: [ 2.00039354  3.0027767  -0.99828491]
Средняя ошибка: 0.001628
